<a href="https://colab.research.google.com/github/Vanagand/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/Unit_2_Sprint_3_Module_2_LESSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

In [0]:
# https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf
# https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html



# Explore tabular data for supervised machine learning 🍌

Wrangling your dataset is often the most challenging and time-consuming part of the modeling process.

In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

Let’s get set up:


In [0]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [0]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [0]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

orders.csv
departments.csv
products.csv
aisles.csv
order_products__prior.csv
order_products__train.csv


### For each csv file, look at its shape & head 

In [0]:
import pandas as pd
from IPython.display import display

def preview():
    for filename in glob('*.csv'):
        df = pd.read_csv(filename)
        print('\n', filename, df.shape)
        display(df.head())

preview()


 orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 products.csv (49688, 4)


...

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation



 order_products__prior.csv (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [0]:
order_products__train = pd.read_csv('order_products__train.csv')

In [0]:
order_products__train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [0]:
# TODO: Add comment
temp = order_products__train.sort_values(by='product_id', ascending=False).head()
temp

,order_id,product_id,add_to_cart_order,reordered
891142,2198380,49688,10,0
1209047,2986153,49688,16,0
269349,655800,49688,10,1
1014644,2508423,49688,3,0
446055,1092104,49687,1,0


In [0]:
# TODO: Add comment
temp.groupby('product_id').count()

,order_id,add_to_cart_order,reordered
product_id,,,
49687,1,1,1
49688,4,4,4


In [0]:
order_products__train.groupby('product_id').order_id.count().sort_values(ascending=False)

product_id
24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
29560        1
8001         1
29566        1
8005         1
24836        1
Name: order_id, Length: 39123, dtype: int64

In [0]:
# Product id 24852 was ordered almost 20 thousand times
# in the orders in this table
order_products__train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [0]:
# Read the products table, to
# figure out the product name for product id 24852
products = pd.read_csv('products.csv')
products[products.product_id==24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [0]:
order_products__train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [0]:
train = pd.merge(order_products__train, products, on='product_id', how='inner')

In [0]:
train['product_name'].value_counts()

Banana                                     18726
Bag of Organic Bananas                     15480
Organic Strawberries                       10894
Organic Baby Spinach                        9784
Large Lemon                                 8135
                                           ...  
Peppercorns, 100% Organic, Black, Whole        1
Pineapple Chunks In Natural Juice              1
Borax 30C Homeopathic Pellets                  1
Simply Vanilla Ice Cream Sandwiches            1
Go  White Chia! Organic Seeds                  1
Name: product_name, Length: 39123, dtype: int64

## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [0]:
train['bananas'] = train['product_name'] == 'Banana'

In [0]:
# 1.4% of the products ordered are (non-organic) bananas
train['bananas'].value_counts(normalize=True)

False    0.986476
True     0.013524
Name: bananas, dtype: float64

In [0]:
train

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,bananas
0,1,49302,1,1,Bulgarian Yogurt,120,16,False
1,816049,49302,7,1,Bulgarian Yogurt,120,16,False
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,False
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,False
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,False
...,...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,Sprinkles Decors,97,13,False
1384613,3420084,47935,20,0,Classic Original Lip Balm SPF 12,73,11,False
1384614,3420084,9491,21,0,Goats Milk & Chai Soap,25,11,False
1384615,3420088,16380,12,0,Stevia Sweetener,97,13,False


In [0]:
# For each order, were any of the products bananas?
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()

In [0]:
# ~ 14% of orders included bananas
train_wrangled['bananas'].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

# Join relational data for supervised machine learning

## Overview
Often, you’ll need to join data from multiple relational tables before you’re ready to fit your models.

## Which customers have ordered bananas before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? 

(If not, how can we combine this information?)

No, we don't have a table with both these id's. But:
- `orders.csv` has `user_id` and `order_id`
- `order_products__prior.csv` has `order_id` and `product_id`
- `order_products__train.csv` has `order_id` and `product_id` too



In [0]:
# In the train table, which orders included bananas?
train[train.bananas==True]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,bananas
129688,226,24852,2,0,Banana,24,4,True
129689,473,24852,2,0,Banana,24,4,True
129690,878,24852,2,1,Banana,24,4,True
129691,1042,24852,1,1,Banana,24,4,True
129692,1139,24852,1,1,Banana,24,4,True
...,...,...,...,...,...,...,...,...
148409,3419531,24852,2,1,Banana,24,4,True
148410,3419542,24852,6,0,Banana,24,4,True
148411,3419629,24852,5,1,Banana,24,4,True
148412,3420088,24852,9,1,Banana,24,4,True


In [0]:
banana_order_ids = train[train.bananas==True].order_id

In [0]:
# Look at the orders table
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3209464,3390613,193506,prior,36,3,11,4.0
1465979,102107,88111,prior,4,6,7,8.0
414240,3342414,24933,prior,4,4,12,19.0
547997,3056094,33024,prior,18,6,14,4.0
440217,853506,26477,prior,5,2,11,8.0


In [0]:
# In the orders table, which orders included bananas?
orders[orders.order_id.isin(banana_order_ids)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
25,1492625,2,train,15,1,11,30.0
485,698604,34,train,6,4,13,30.0
623,1864787,43,train,12,5,11,26.0
652,1947848,46,train,20,4,20,11.0
881,13318,56,train,13,1,9,6.0
...,...,...,...,...,...,...,...
3420581,681092,206181,train,15,1,13,30.0
3420692,2530125,206191,train,6,0,9,30.0
3420786,3383615,206196,train,5,0,15,30.0
3420933,1716008,206205,train,4,1,16,10.0


In [0]:
# Check an order_id from above, confirm that yes it includes bananas
train[train.order_id==1492625]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,bananas
40632,1492625,11913,28,0,Shelled Pistachios,117,19,False
137883,1492625,24852,8,1,Banana,24,4,True
199519,1492625,9387,10,0,Granny Smith Apples,24,4,False
202785,1492625,45066,9,1,Honeycrisp Apple,24,4,False
241750,1492625,5450,11,1,Small Hass Avocado,24,4,False
269520,1492625,33957,21,1,Mexican Casserole Bowl,38,1,False
286130,1492625,41787,5,1,Bartlett Pears,24,4,False
356655,1492625,22963,1,1,Organic Roasted Turkey Breast,96,20,False
503295,1492625,48523,25,0,Birthday Cake Light Ice Cream,37,1,False
523860,1492625,38547,13,0,Bubblegum Flavor Natural Chewing Gum,46,19,False


In [0]:
banana_orders = orders[orders.order_id.isin(banana_order_ids)]

In [0]:
# In the orders table, which users have bought bananas?
banana_user_ids = banana_orders.user_id.unique()

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [0]:
# orders table, shape before getting subset
orders.shape

(3421083, 7)

In [0]:
# orders table, shape after getting subset
orders_subset = orders[orders.user_id.isin(banana_user_ids)]
orders_subset.shape

(332661, 7)

## What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

- Products per order
- Time of day
- Have they reordered bananas before? (Have ordered bananas >= 2 times)
- Other fruit they buy 
- Size of orders (customers with smaller orders on average are less likely to be reordering any particular product on their next order)


- Frequency of banana orders:
    - % of orders 
    - Time between banana orders: Every n days on average
    - Raw count: Total orders, how many times have you ordered bananas?

- Recency of banana orders
    - n days since you ordered banana




In [0]:
# Pick out an individual user for exploration & experiments
USER_ID = 2
orders_subset[orders_subset.user_id==USER_ID]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
11,2168274,2,prior,1,2,11,NaN
12,1501582,2,prior,2,5,10,10.0
13,1901567,2,prior,3,1,10,3.0
14,738281,2,prior,4,2,10,8.0
15,1673511,2,prior,5,3,11,8.0
16,1199898,2,prior,6,2,9,13.0
17,3194192,2,prior,7,2,12,14.0
18,788338,2,prior,8,1,15,27.0
19,1718559,2,prior,9,2,9,8.0
20,1447487,2,prior,10,1,11,6.0


## Challenge

**Continue to clean and explore your data.** Can you **engineer features** to help predict your target? For the evaluation metric you chose, what score would you get just by guessing? Can you **make a fast, first model** that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint. But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset today. Follow the instructions in the assignment notebook. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!